In [1]:
import gpt as g
import numpy as np

SharedMemoryNone: SharedMemoryAllocate 1073741824 GPU implementation 
0SharedMemoryNone:  SharedMemoryNone.cc acceleratorAllocDevice 1073741824bytes at 0x7f0396200000 for comms buffers 

__|__|__|__|__|__|__|__|__|__|__|__|__|__|__
__|__|__|__|__|__|__|__|__|__|__|__|__|__|__
__|_ |  |  |  |  |  |  |  |  |  |  |  | _|__
__|_                                    _|__
__|_   GGGG    RRRR    III    DDDD      _|__
__|_  G        R   R    I     D   D     _|__
__|_  G        R   R    I     D    D    _|__
__|_  G  GG    RRRR     I     D    D    _|__
__|_  G   G    R  R     I     D   D     _|__
__|_   GGGG    R   R   III    DDDD      _|__
__|_                                    _|__
__|__|__|__|__|__|__|__|__|__|__|__|__|__|__
__|__|__|__|__|__|__|__|__|__|__|__|__|__|__
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  


Copyright (C) 2015 Peter Boyle, Azusa Yamaguchi, Guido Cossu, Antonin Portelli and other authors

This program is free software; you can redistribute it and/or modify
it under the t

## Lecture 2: Linear algebra with lattices

We start by creating a $4^4$ double precision grid.

In [2]:
grid = g.grid([4, 4, 4, 4], g.double)

Next, we create a parallel pseudorandom number generator and two color vectors from a complex normal distribution.

In [3]:
rng = g.random("seed for rng")
v1 = g.vcolor(grid)
v2 = g.vcolor(grid)
rng.cnormal([v1,v2]);

GPT :       0.409328 s : Initializing gpt.random(seed for rng,vectorized_ranlux24_389_64) took 0.000268459 s


We inspect a one-dimensional slice of $v_1$.

In [4]:
v1[0,0,0,:]

array([[-0.58527248-0.85360387j, -1.580872  -1.50785674j,
        -1.04090035+0.40186692j],
       [-0.23550862-0.90891595j,  0.73888871+0.21903038j,
         1.10074976+1.52119033j],
       [ 0.79255516+0.64832885j, -0.22812566+0.54092838j,
         1.17416062-0.72316771j],
       [-0.81076797+0.6141707j ,  0.6581139 -0.02166552j,
         0.17446881+0.19891321j]])

Let us now take a linear combination of $\frac12 v_1 + 3 v_2$.  Note that there is a difference between abstract expressions and evaluating them to lattice fields.

In [5]:
expr = 1./2. * v1 + 3. * v2

g.message("This is an expression:", expr)

result = g.eval(expr)

g.message("And this is the coordinate (0,0,0,0) of the evaluated lattice object:", result[0,0,0,0])

GPT :       0.451677 s : This is an expression:  + ((0.5+0j))*lattice(ot_vector_color(3),double) + ((3+0j))*lattice(ot_vector_color(3),double)
GPT :       0.468984 s : And this is the coordinate (0,0,0,0) of the evaluated lattice object: tensor([ 0.09215363+4.58520759j  0.53058119-4.0223991j  -0.79159505+2.5800456j ],ot_vector_color(3))


Since the "=" operator in Python cannot be overloaded and always assigns the Python object on the right side to the symbol to left, GPT uses the operator "@=" to assign the result of an expression to a lattice field.

In [6]:
result2 = g.lattice(v1) # creates a lattice of same type than v1 without initializing the values
result2 @= expr

g.message("Difference between g.eval(...) and @= operation:", g.norm2(result2 - result))

GPT :       0.478653 s : Difference between g.eval(...) and @= operation: 0.0


It is also useful to know of the increment operator:

In [7]:
result += v1

Next, let us construct site-local inner and outer products from our vectors.

In [8]:
local_inner_product = g.complex(grid)
local_inner_product @= g.adj(v1) * v2

local_outer_product = g.mcolor(grid)
local_outer_product @= v1 * g.adj(v2)

g.message("Origin of inner_product field:", local_inner_product[0,0,0,0])

g.message("Origin of outer_product field:", local_outer_product[0,0,0,0])


GPT :       0.504401 s : Origin of inner_product field: (-0.14170995506280581+0.7288503091897771j)
GPT :       0.510657 s : Origin of outer_product field: tensor([[-1.50115922+0.86831104j  0.67227476-1.01352379j -0.6240419 +0.54129306j]
                       :  [-2.72189862+2.44771251j  0.94667551-2.38631285j -1.05290498+1.38997313j]
                       :  [ 0.53787765+1.79054559j -0.89617918-0.95709307j  0.41277376+0.7891515j ]],ot_matrix_su_n_fundamental_group(3))


In [9]:
v12 = g.inner_product(v1, v2)
v11 = g.inner_product(v1, v1)
n1 = g.norm2(v1)

g.message("v1 . v2", v12)
g.message("sum(local_inner_product)", g.sum(local_inner_product))

g.message("v1 . v1", v11)
g.message("norm2(v1)", n1)


GPT :       0.521912 s : v1 . v2 (-44.827357722843956-52.851620669240525j)
GPT :       0.524068 s : sum(local_inner_product) (-44.82735772284396-52.851620669240525j)
GPT :       0.525660 s : v1 . v1 (1592.0538142528967-2.14659970599242e-15j)
GPT :       0.527248 s : norm2(v1) 1592.0538142528967


We move on to SU(3) matrices now.  g.mcolor is short-hand for the QCD gauge group, but GPT has general SU(N) and U(1) implemented.  We initialize a SU(3) matrix with a random near-unit element and then compute its site-wise determinant and inverse and check the results.

In [29]:
V = g.mcolor(grid)
rng.normal_element(V, scale=0.01)

g.message("Origin of V:", V[0,0,0,0])

GPT :     277.981400 s : Origin of V: tensor([[ 9.99958757e-01-0.00101886j -7.76748844e-04+0.00360413j
                       :   -9.24397517e-04+0.0081853j ]
                       :  [ 7.82807158e-04+0.00363416j  9.99961810e-01-0.0061684j
                       :    4.02449857e-03+0.00288309j]
                       :  [ 8.60354990e-04+0.00817842j -4.05774034e-03+0.00287582j
                       :    9.99927990e-01+0.00718697j]],ot_matrix_su_n_fundamental_group(3))


In [30]:
det_V = g.matrix.det(V)

g.message("Slice of local determinant:", det_V[0,0,0,:])

GPT :     278.380816 s : Slice of local determinant: [[1.+1.56076289e-18j]
                       :  [1.-2.61415044e-18j]
                       :  [1.+1.35209701e-18j]
                       :  [1.+1.11941845e-18j]]


In [31]:
inv_V = g.matrix.inv(V)

g.message("Difference between matrix inverse and adjoint for unitary matrix:", g.norm2(inv_V - g.adj(V)))

GPT :     278.708010 s : Difference between matrix inverse and adjoint for unitary matrix: 1.6731290387825207e-29


We can also check that the logarithm of the matrix is anti-Hermitian.

In [32]:
logV = g.matrix.log(V)
g.norm2(logV + g.adj(logV))

2.0041293724770068e-29

There are also component-wise operations, see, e.g., the cosine:

In [36]:
g.component.cos(V)[0,0,0,0]

tensor([[0.54033729+8.57321741e-04j 1.00000619+2.79950666e-06j
  1.00003307+7.56655425e-06j]
 [1.0000063 -2.84484965e-06j 0.54034472+5.19043355e-03j
  0.99999606-1.16029684e-05j]
 [1.00003307-7.03642540e-06j 0.9999959 +1.16693173e-05j
  0.54037685-6.04739656e-03j]],ot_matrix_su_n_fundamental_group(3))